In [ ]:
#importando as dependencias necessarias
import pandas as pd
import json, csv, glob
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr, spearmanr
import itertools
import time, datetime, os

In [ ]:
#json nao esta mais sendo utilizado
"""formatando o arquivo de logger para ser compativel com o pandas
#obs: o arquivo de logger sera sobreescrito (podendo nao ser necesssario executar essa celula para determinado arquivo)
dic = {'ID_smartwatch':[], 'dateTime':[], 'battery':[], 'fileSize':[], 'pressure':[], 'lux':[], 'heartRate':[],
         'ecg':[], 'ppg':[], 'saturation':[], 'accelerometerZ':[], 'accelerometerX':[],
         'accelerometerY':[], 'gyroscopeZ':[], 'gyroscopeX':[], 'gyroscopeY':[]}

for filename in os.listdir("loggers"):
    print(filename)
    with open('loggers/'+filename, 'r') as file:
        d = file.read()
        f = json.loads(d)
        for key in f:
            dic[key] = dic[key] + list(f[key][0][1:-1].split(', '))
        
dic['ID_smartwatch'] = dic['ID_smartwatch'][0]
with open('data/logger.json', 'w') as file:
    file.write(json.dumps(dic))

with open('data/loggerECG.json', 'r+') as file:
    dic = file.read()
    f = json.loads(dic)
    for key in f:
        f[key] = list(f[key][0][1:-1].split(', '))

    for key in f:
        d = {}
        for i in range(len(f[key])):
            d[i] = f[key][i]
        f[key] = d
    file.seek(0)
    file.write(json.dumps(f))
    file.truncate()
"""

In [ ]:
#para um unico arquivo.bin
with open('loggers/coletasJuntas.bin', 'r') as file:
    f = csv.reader(file)
    header = next(f)
    newHeader = [i.strip() for i in header[0:-1]]
    idSmart = header[-1][11:]
    print(newHeader, idSmart)

    with open('data/log.csv', 'w') as file:
        w = csv.writer(file)
        w.writerow(newHeader)
        for row in f:
            w.writerow(row)

In [ ]:
#para multiplos arquivos.bin
filenames = []
i = 0

for filename in os.listdir("loggers"):
    filenames.append(filename)
    
    with open('loggers/'+filename, 'r') as file:
        f = csv.reader(file)
        header = next(f)
        if not i:
            idSmart = header[-1][11:]
            header = [j.strip() for j in header[0:-1]] 
            print(header, idSmart)
            
        if not i:
            with open('data/log2.csv', 'w') as new_file:
                w = csv.writer(new_file)
                w.writerow(header)
                for row in f:
                    w.writerow(row)
        else:
            with open('data/log2.csv', 'a') as new_file:
                w = csv.writer(new_file)
                for row in f:
                    w.writerow(row)
    i +=1

In [ ]:
#definicao de variaveis
items = ['battery', 'fileSize', 'pressure', 'lux', 'heartRate',
         'ecg', 'ppg', 'saturation', 'accelerometerZ', 'accelerometerX',
         'accelerometerY', 'gyroscopeZ', 'gyroscopeX', 'gyroscopeY']
units = {'battery': 'bateria (%)',
             'fileSize': 'armazenamento (MB)',
             'pressure': 'pressao (hPa)',
             'lux': 'luminosidade (lux)',
             'heartRate': 'taxa de batimentos (bpm)',
             'ecg': 'ecg cru (mV)',
             'ppgGreen': 'ppg cru (mV)',
             'ppgIR': 'ppg cru (mV)',
             'ppgRed': 'ppg cru (mV)',
             'saturation': 'oxigenacao crua (mV)',
             'accelerometerZ': 'aceleracao (m/s²)',
             'accelerometerX': 'aceleracao (m/s²)',
             'accelerometerY': 'aceleracao (m/s²)',
             'gyroscopeZ': 'giroscopio (rad/s)',
             'gyroscopeX': 'giroscopio (rad/s)',
             'gyroscopeY': 'giroscopio (rad/s)',
             'sleepiness': 'nivel de sono'}

In [ ]:
#json nao esta mais sendo utilizado
"""
#Processamento dos dados para arquivos json
df = pd.read_json('data/logger.json')
#removendo o id do smartwatch do dataframe
df.drop(['ID_smartwatch'], axis=1, inplace=True)
df.head()
#print(df)
"""

In [ ]:
#Processamento dos dados caso arquivo seja em csv
df = pd.read_csv('data/log.csv')
df.head()

In [ ]:
#visao geral dos dados (quantidade, media, desvio padrao, minimo, q1, mediana, q3 e maximo)
print(df.sleepiness.value_counts())
df.describe()

In [ ]:
#checando a existencia de valores nulos/faltantes
#df.isna().sum()
df.info()

In [ ]:
"""cols = list(df.columns)
for col in cols:
    if df[col].dtype == 'float64':
        df[col] = df[col].astype('int64')
        print(df[col].dtype)"""

In [ ]:
#convertendo datetime para valor numerico
time = []
y, mo, d = map(int, str(df['dateTime'][0])[0:10].split('-'))
h,m,s = map(int, str(df['dateTime'][0])[11:19].split(':'))
startingTime = datetime.datetime(y, mo, d, h, m, s).timestamp()

for val in range(len(df.dateTime)):
    v = str(df.dateTime[val])
    y, mo, d = map(int, v[0:10].split('-'))
    h,m,s = map(int, v[11:19].split(':'))
    ts = datetime.datetime(y, mo, d, h, m, s).timestamp()
    ts = ts - startingTime
    time.append(ts)
    
df.drop('dateTime', axis = 1, inplace=True)
df['dateTime'] = time
df.head()

In [ ]:
#substituindo leituras erroneas por NaN
df.ppgGreen.replace(0, np.nan, inplace=True)
df.ppgRed.replace(0, np.nan, inplace=True)
df.heartRate.replace(0, np.nan, inplace=True)
df.ppgIR.replace(0, np.nan, inplace=True)
df.saturation.replace(0, np.nan, inplace=True)
hm = sns.heatmap(df.isnull(),cbar=False,yticklabels=False,cmap = 'Blues')
figure = hm.get_figure()
figure.savefig('graphs/missing.png', bbox_inches='tight', dpi=800)

In [ ]:
#substituindo os valores faltantes pela media da coluna (caso nao existam nao e necessario executar essa celula)
#obs: apenas valido para valores numericos
num_col = ['heartRate', 'ppgRed', 'ppgGreen', 'ppgIR', 'saturation'] # lista com as colunas que possuem valores faltantes
print(df.isna().sum())
for col in num_col:
    df[col]=pd.to_numeric(df[col])
    df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
#visualizando as leituras de cada variavel
items = ['pressure', 'lux', 'heartRate', 'ppgGreen', 'ppgRed', 'ppgIR', 'saturation', 'accelerometerZ', 'accelerometerX',
         'accelerometerY', 'gyroscopeZ', 'gyroscopeX', 'gyroscopeY', 'sleepiness']
for var in items:
    y = df[var]
    #x = range(5000)
    x = range(y.size)
    plt.figure()
    plt.title(f"{var} x index")
    plt.xlabel("Leituras")
    plt.ylabel(f"{units[var]}")
    plt.plot(x,y)
    plt.savefig(f'graphs/linePlot_{y.name}.png', bbox_inches='tight')

In [ ]:
for var in items:
    y = list(df[var])
    plt.figure()
    plt.title(f"{var} boxplot")
    plt.ylabel(f"{units[var]}")
    plt.boxplot(y)
    plt.savefig(f'graphs/boxplot_{var}.png', bbox_inches='tight')

In [ ]:
#visualizando a distribuicao dos dados
for var in items:
    #nCols = len(df.columns)
    #nRows = df.dateTime.size
    #plt.subplot(nRows+1,nCols,1)
    plt.figure()
    plt.title(f"{var} distribution")
    sns.distplot(df[var],kde=True)
    plt.savefig(f'graphs/distribution_{var}.png', bbox_inches='tight')

In [ ]:
#calculando a covariancia entre variaveis
covariance = np.cov([df.ppgGreen, df.heartRate, df.lux])
print(covariance)

In [ ]:
#normalizando a covariancia para que se torne interpretavel(correlacao de pearson)
#metodo mais apropriado para distrubuicoes gaussianas
items = [df.ppgGreen, df.heartRate, df.lux]
for pair in itertools.combinations(items, 2):
    pCorr, _ = pearsonr(*pair)
    print(f"A correlacao de Pearson entre {pair[0].name} e {pair[1].name} eh: {pCorr:.3f}")

In [ ]:
#interpretando a covariancia pela correlacao de spearman (para distribuicoes nao gaussianas)
items = [df.ppgGreen, df.heartRate, df.lux]
for pair in itertools.combinations(items, 2):
    sCorr, _ = spearmanr(*pair)
    print(f"A correlacao de Spearman entre {pair[0].name} e {pair[1].name} eh: {sCorr:.3f}")

In [ ]:
#ddf = df.copy()
#ddf.drop(['saturation'], axis=1, inplace=True)
cor = sns.heatmap(df.corr(method='spearman'),cmap='Blues',annot=True, fmt='.2f', annot_kws={"fontsize":7})
figure = cor.get_figure()    
figure.savefig('graphs/correlation_spearman.png', bbox_inches='tight', dpi=800)

In [ ]:
#comparacao de variavel com classificador por scatter
compare = ["saturation", "ppgRed", "pressure"]
for var in items:
    plt.figure()
    plt.title(f"{var} x sleepiness")
    plt.scatter(x=var, y='sleepiness', data=df)
    plt.xlabel(f'{var}')
    plt.ylabel('sleepiness')
    plt.savefig(f'graphs/scatter_{var}.png', bbox_inches='tight')

In [ ]:
#comparacao de variavel com classificador por boxplot
compare = ["saturation", "ppgRed", "pressure"]
for var in compare:
    plt.figure()
    plt.title(f"{var} x sleepiness")
    sns.boxplot(x='sleepiness',y=var,data=df)
    plt.ylabel(f'{var}')
    plt.xlabel('sleepiness')
    plt.savefig(f'graphs/boxComparison_{var}.png', bbox_inches='tight')


In [ ]:
#normalizacao por maximum absolute scaling
def maxAbsoluteScale(df):
    scaledDf = df.copy()
    #Divide cada valor de uma coluna pelo valor maximo da mesma
    for column in scaledDf.columns:
        #as colunas de horario, tamanho de arquivo e bateria sao desconsideradas
        if column != 'dateTime' and column != 'size' and column != 'battery':
            scaledDf[column] = scaledDf[column]  / scaledDf[column].abs().max()
    #retorna os valores em uma escala de -1 a 1
    return scaledDf

In [ ]:
#normalizacao por min-max
def minMaxScale(df):
    scaledDf = df.copy()
    #aplicando a formula do min-max (x-min / range)
    for column in scaledDf.columns:
        #as colunas de horario, tamanho de arquivo e bateria sao desconsideradas
        if column != 'dateTime' and column != 'size' and column != 'battery':
            scaledDf[column] = (scaledDf[column] - scaledDf[column].min()) / (scaledDf[column].max() - scaledDf[column].min())
    #retorna os valores em uma escala entre 0 e 1
    return scaledDf

In [ ]:
#normalizacao por z-score (standardize)
def standardize(df):
    scaledDf = df.copy()
    #aplicando a formula do z-score (x-media / desvio padrao)
    for column in scaledDf.columns:
        #as colunas de horario, tamanho de arquivo e bateria sao desconsideradas
        if column != 'dateTime' and column != 'size' and column != 'battery':
            scaledDf[column] = (scaledDf[column] - scaledDf[column].mean()) / scaledDf[column].std()
    #retorna os valores com media = 0 e desvio padrao = 1
    return scaledDf

In [ ]:
#normalizacao por robust scaling (ideal para dados com muitos outliers)
def robustScaling(df):
    scaledDf = df.copy()
    #aplicando a formula do robust scaling (x-mediana/(q3-q2)*) *interquartille range
    for column in scaledDf.columns:
        #as colunas de horario, tamanho de arquivo e bateria sao desconsideradas
        if column != 'dateTime' and column != 'size' and column != 'battery':
            scaledDf[column] = (scaledDf[column] - scaledDf[column].median())  / (scaledDf[column].quantile(0.75) - scaledDf[column].quantile(0.25))
    return scaledDf

In [ ]:
#calculando o numero de outliers
def outlierCount(df, var):
    try:
        q1= df[var].quantile(0.25)
        q3 = df[var].quantile(0.75)
        iqRange = q3 - q1
        maxVal = q3 + 1.5 * iqRange
        minVal = q1 - 1.5 * iqRange
        count = len(df[(df[var] < minVal)]) + len(df[(df[var] > maxVal)])
        return count, minVal, maxVal
    except:
        #print(f"{var} falhou")
        return "null", "null", "null"

for col in df.columns:
    a, b, c = outlierCount(df, col)
    print(f'{col}: {a},'.ljust(25) + f' |min = {b},'.ljust(35) + f' |max = {c}')

In [ ]:
#tratando outliers
def removeOutliers(df, *args):
    newDf = df.copy()
    for var in args:
        count, minVal, maxVal = outlierCount(df, var)
        newDf = newDf[(newDf[var] > minVal) & (newDf[var] < maxVal)]
    return newDf
    
noOutliers = removeOutliers(df, 'heartRate')
print(outlierCount(noOutliers, 'heartRate'))

In [ ]:
maxAbsDf = maxAbsoluteScale(df)
minMaxDf = minMaxScale(df)
zScoreDf = standardize(df)
robustDf = robustScaling(df)

In [ ]:
y = list(noOutliers.heartRate)
plt.boxplot(y)

In [ ]:
y = df.heartRate
x = range(0, len(df.dateTime))
plt.plot(x,y)

In [ ]:
y = zScoreDf.heartRate
x = zScoreDf.dateTime
plt.plot(x,y)

In [ ]:
zScoreNoOut = standardize(noOutliers)
y = zScoreNoOut.ppgGreen
x = zScoreNoOut.dateTime
plt.plot(x,y)

In [ ]:
#checando a correlacao apos processamento
items = [zScoreNoOut.ecg, zScoreNoOut.heartRate, zScoreNoOut.lux]
for pair in itertools.combinations(items, 2):
    sCorr, _ = spearmanr(*pair)
    print(f"A correlacao de Spearman entre {pair[0].name} e {pair[1].name} eh: {sCorr:.3f}")